## Pivoting and unpivoting
By the end of this lecture you will be able to:
- make a `DataFrame` wide with `pivot`
- make a `DataFrame` long with `unpivot`

In [ ]:
import polars as pl
import numpy as np

## Pivot
We start with a simple example of some data with sales from a bikeshop

In [ ]:
sales_data = pl.DataFrame({
    'date': ['2022-01-01', '2022-01-02', '2022-01-01', '2022-01-02','2022-01-03'],
    'region': ['East', 'West', 'East', 'West','West'],
    'bike_type': ['Mountain', 'Mountain', 'Road', 'Road','Mountain'],
    'sales': [100, 200, 300, 400,500]
})
sales_data

We want to pivot the data so that we have:
- a row for each date as the `index`
- a column that we pivot `on` for each product and
- the sales `values` for that product and date 

In [ ]:
(
    sales_data
    .pivot(
        index='date', 
        on='bike_type', 
        values='sales'
    )
)

When we use `pivot` we say that we turn a `DataFrame` from **long** to **wide** format. 

What happens in the cells where there is no corresponding `date` and `bike_type` values in the original `DataFrame`? In this case Polars inserts a `null` value.

What happens in the cells where there are multiple values with the same `date` and `bike_type`? In this case Polars aggregates the values using the `aggregate_function` argument. In this example we take the `mean` of these values

In [ ]:
(
    sales_data
    .pivot(
        index='date', 
        on='bike_type', 
        values='sales',
        aggregate_function="mean"
    )
)

We don't have to pass the `aggregate_function` argument if there are no aggregations in the data, but Polars raises an `Exception` if an aggregation is required and `aggregate_function` has not been passed. We look at aggregations more below.

We can also create new columns from data in multiple original columns by passing a list of column names to the `columns` argument

In [ ]:
(
    sales_data
    .pivot(
        index='date', 
        on=['region','bike_type'], 
        values='sales',
        aggregate_function="first"
    )
)

The column names are set out in a JSON-esque string with `{}`. These are just strings though as we see here where we select one of the columns

In [ ]:
(
    sales_data.pivot(
        index="date",
        on=["region", "bike_type"],
        values="sales",
        aggregate_function="first",
    )
    .select("date", '{"East","Mountain"}')
)

### Pivots and aggregation
When there are multiple values in the original `DataFrame` that correspond to a position in the pivoted `DataFrame` then Polars must aggregate them.

We tell Polars how to aggregate them using the `aggregate_function` argument. We demonstrate this by pivoting by `region`.

In our original `DataFrame` we have two values for each region on 2022-01-01 and 2022-01-02

In [ ]:
sales_data

We can alternatively specify one of the following aggregation strings: `sum`, `max`, `min`, `mean`, `median`, `last`, `count`.

We can also supple a custom aggregation expression using `pl.element` (see the List dtype lectures if you have not come across `pl.element` before). In this example we aggregate by taking the 75% percentile of the values

In [ ]:
(
    sales_data
    .pivot(
        index='date', 
        on='region', 
        values='sales',
        aggregate_function=pl.element().quantile(0.75,interpolation="linear")
    )
)

The pivoted columns are ordered by the order Polars finds them in the original `on` column - so in this case there was an `East` entry before a `West` entry.

We can ensure the columns are ordered lexically with the `sort_columns` argument

In [ ]:
(
    sales_data
    .pivot(
        index='date', 
        on='region', 
        values='sales',
        aggregate_function="mean",
        sort_columns=True
    )
)

### Pivot in lazy mode?
In lazy mode Polars must know the schema (column names and dtypes) at each stage of a query plan. However, after a `pivot` the column names cannot be known in advance as they are dependant on the data. As such a `pivot` is not - and will not - be available in lazy mode.  

If you have a lazy query but want to do a `pivot` then you can:
- `collect` your query, do the `pivot` and then call `lazy` to resume in lazy mode

## Unpivoting from wide to long
To convert a `DataFrame` from wide to long we use the `unpivot` method (which used to be called `melt` as in Pandas).

We begin this example with a wide `DataFrame` that we get from calling `pivot` on `sales_data`

In [ ]:
sales_pv = (
    sales_data
    .pivot(
        index='date', 
        on='bike_type', 
        values='sales',
        aggregate_function="mean"
    )
)
sales_pv

We unpivot the `DataFrame` by specifying:
- which metadata column(s) will identify the data on each row in `index`
- which columns will provide the values that we are unpivoting `on`

In [ ]:
(
    sales_pv
    .unpivot(
        index="date",
        on=["Mountain","Road"]
    )
)

The column names in `on` become the data in the `variable` column.

If we want to use all columns that are not specified in the `index` we can just omit the `on` argument

In [ ]:
(
    sales_pv
    .unpivot(
        index="date",
    )
)

We can optionally specify different names for the `variable` and `value` column with the `variable_name` and `value_name` arguments

In [ ]:
(
    sales_pv
    .unpivot(
        index="date",
        variable_name="bike_type",
        value_name="sales"
    )
)

### Unpivot in lazy mode?
We can use `unpivot` in lazy mode as the new column names along with their dtypes can be known in advance.

### Unstacking
Another way to transform from long to wide format is with `unstack`.  Unlike a `pivot` where Polars first does a `group_by` to get the pivot keys the `unstack` method works off an integer `step` argument.

When, for example, `step=2` Polars:
- goes through the column and finds the first two values
- creates the first column of the new `DataFrame` from these two values
- gets the next two values in that column
- creates anpther column with these two values
- repeats this process for each column

In [ ]:
(
    sales_pv
    .unpivot(
        index="date",
    )
    .unstack(step=2,how="vertical")
)

An `unstack` is typically faster than a `pivot` because it does not need do an internal `group_by` to find which values go in which columns.

The `unstack` direction can also be horizontal

In [ ]:
(
    sales_pv
    .unpivot(
        index="date",
    )
    .unstack(step=2,how="horizontal")
)

We can use `unstack` instead of `pivot` when we know that we can provide the `step` integer value to do the pivoting

## Exercises
In the exercises you will develop your understanding of:
- converting a `DataFrame` to wide format with `pivot`
- converting a `DataFrame` to long format for visualisation with `unpivot`

### Exercise 1
For this exercise we use a dataset of bike sales in different countries.

We derive a `year` column from the `date` - see the lecture on Extracting datetime components in the Time Series section for more on this.

In [ ]:
sales_df = (
    pl.read_parquet("../data/bike_sales.parquet")
    .with_columns(
        pl.col("date").dt.year().alias("year")
    )
)
sales_df.head(3)

Pivot the data to have a year on each row and a column for each `sub category`. Aggregate by getting the sum of the `order quantity`. Ensure the years are in ascending order

In [ ]:
(
    sales_df
    <blank>
)

We want to visualise this data as a time series with Plotly so melt the pivoted `DataFrame` and assign it to `annual_sales_df`

In [ ]:
annual_sales_df = (
    sales_df
    <blank>
)

We can now plot the output using `px.line` in Plotly (feel free to do this with your preferred visualisation library). If you haven't come across Plotly before see the lecture in the Visualisation section.

In [ ]:
import plotly.express as px
px.line(
    <blank>
)

### Exercise 2
In this exercise we want to identify which words are present in a set of texts. This is a common task in natual language processing often carried out using the CountVectorizer in Scikit-learn.

We begin by defining our `fake_news_df`

In [ ]:
fake_news_df = pl.DataFrame({
    'publication': ['The Daily Deception', 'Faux News Network', 'The Fabricator', 'The Misleader', 
                     'The Hoax Herald', ],
    'date': ['2022-01-01', '2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06', 
             ],
    'title': ['Scientists Discover New Species of Flying Elephant', 
              'Aliens Land on Earth and Offer to Solve All Our Problems', 
              'Study Shows That Eating Pizza Every Day Leads to Longer Life', 
              'New Study Finds That Smoking is Good for You', 
              "World's Largest Iceberg Discovered in Florida"],
    'text': ['In a groundbreaking discovery, scientists have found a new species of elephant that can fly. The flying elephants, which were found in the Amazon rainforest, have wings that span over 50 feet and can reach speeds of up to 100 miles per hour. This is a game-changing discovery that could revolutionize the field of zoology.',
             'In a historic moment for humanity, aliens have landed on Earth and offered to solve all our problems. The extraterrestrial visitors, who arrived in a giant spaceship that landed in Central Park, have advanced technology that can cure disease, end hunger, and reverse climate change. The world is waiting to see how this incredible offer will play out.',
             'A new study has found that eating pizza every day can lead to a longer life. The study, which was conducted by a team of Italian researchers, looked at the eating habits of over 10,000 people and found that those who ate pizza regularly lived on average two years longer than those who didn\'t. The study has been hailed as a breakthrough in the field of nutrition.',
             'In a surprising twist, a new study has found that smoking is actually good for you. The study, which was conducted by a team of British researchers, looked at the health outcomes of over 100,000 people and found that those who smoked regularly had lower rates of heart disease and cancer than those who didn\'t. The findings have sparked controversy among health experts.',
             'In a bizarre turn of events, the world\'s largest iceberg has been discovered in Florida. The iceberg, which is over 100 miles long and 50 miles wide, was found off the coast of Miami by a group of tourists on a whale-watching tour. Scientists are baffled by the discovery and are scrambling to figure out how an iceberg of this size could have']
})
fake_news_df

Begin by:
- converting the text to lowercase and splitting the text by whitespace
- adding a new column called `placeholder` with 1 as a placeholder value

In [ ]:
(
    fake_news_df
    .with_columns(
        <blank>
)

Explode the lists in the `text` column

Pivot the output so that the article metadata is preserved on each row and the remainder of the columns indicate if the column name is present in the text of that article. Ensure the column names are sorted

Replace the `null` values with 0

### Exercise 3

We have a table showing electricity rates paid by a household. The rates can vary by:
- day of the week
- time of day with 00:00 overnight and 12:00 during the day

We identify each household with an `ID`. We use a single household in this exercise.

We store these rates in a table as follows

In [ ]:
pl.Config.set_tbl_rows(14)
df_rates = (
    pl.DataFrame(
        {
            "id":['A','A'],
            "Mon":[1,None],
            "Tue":[1,None],
            "Wed":[1,None],
            "Thu":[1,None],
            "Fri":[1,None],
            "Sat":[None,1],
            "Sun":[None,1],
            "00:00":[0.1,0.3],
            "12:00":[0.15,0.25]
            
        }
    )
)
df_rates

Use `unpivot` and `pivot` (and other methods) to transform `df_rates` to the following `DataFrame` where each row is a (day-of-week,time-of-day) pair and each column is the rate for each `id` in that period

In [ ]:
target_df = pl.DataFrame(
    [
        {"weekday": "Mon", "variable": "00:00", "A": 0.1},
        {"weekday": "Tue", "variable": "00:00", "A": 0.1},
        {"weekday": "Wed", "variable": "00:00", "A": 0.1},
        {"weekday": "Thu", "variable": "00:00", "A": 0.1},
        {"weekday": "Fri", "variable": "00:00", "A": 0.1},
        {"weekday": "Sat", "variable": "00:00", "A": 0.3},
        {"weekday": "Sun", "variable": "00:00", "A": 0.3},
        {"weekday": "Mon", "variable": "12:00", "A": 0.15},
        {"weekday": "Tue", "variable": "12:00", "A": 0.15},
        {"weekday": "Wed", "variable": "12:00", "A": 0.15},
        {"weekday": "Thu", "variable": "12:00", "A": 0.15},
        {"weekday": "Fri", "variable": "12:00", "A": 0.15},
        {"weekday": "Sat", "variable": "12:00", "A": 0.25},
        {"weekday": "Sun", "variable": "12:00", "A": 0.25},
    ]
)
target_df

See the hint below to get started if you get stuck. The solution is also broken into steps below

In [ ]:
#Hint:
#- transform the days of the week to long format first

In [ ]:
(
    df_rates
)

## Solutions

### Solution to exercise 1

In [ ]:
sales_df = (
    pl.read_parquet("../data/bike_sales.parquet")
    .with_columns(
        pl.col("date").dt.year().alias("year")
    )
)
sales_df.head(3)

Pivot the data to have a year on each row and a column for each `sub category` of bike. Aggregate by getting the sum of the `order quantity`. Ensure the years are in ascending order

In [ ]:
(
    sales_df
    .pivot(
        index="year",
        on="sub category",
        values="order quantity",
        aggregate_function="sum",
    )
    .sort("year")
)

We want to visualise this data as a time series with Plotly so unpivot the pivoted `DataFrame` and assign it to `annual_sales_df`

In [ ]:
annual_sales_df = (
    sales_df
    .pivot(
        index="year",
        on="sub category",
        values="order quantity",
        aggregate_function="sum"
    )
    .sort("year")
    .unpivot(
        index="year",
    )
)

Plot the output using `px.line`

In [ ]:
import plotly.express as px
px.line(
    x=annual_sales_df["year"],
    y=annual_sales_df["value"],
    color=annual_sales_df["variable"]
)

### Solution to exercise 2
In this exercise we want to identify which words are present in a set of texts. This is a common task in natual language processing often carried out using the CountVectorizer in Scikit-learn.

We begin by defining our `fake_news_df`

In [ ]:
fake_news_df = pl.DataFrame({
    'publication': ['The Daily Deception', 'Faux News Network', 'The Fabricator', 'The Misleader', 
                     'The Hoax Herald', ],
    'date': ['2022-01-01', '2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06', 
             ],
    'title': ['Scientists Discover New Species of Flying Elephant', 
              'Aliens Land on Earth and Offer to Solve All Our Problems', 
              'Study Shows That Eating Pizza Every Day Leads to Longer Life', 
              'New Study Finds That Smoking is Good for You', 
              "World's Largest Iceberg Discovered in Florida"],
    'text': ['In a groundbreaking discovery, scientists have found a new species of elephant that can fly. The flying elephants, which were found in the Amazon rainforest, have wings that span over 50 feet and can reach speeds of up to 100 miles per hour. This is a game-changing discovery that could revolutionize the field of zoology.',
             'In a historic moment for humanity, aliens have landed on Earth and offered to solve all our problems. The extraterrestrial visitors, who arrived in a giant spaceship that landed in Central Park, have advanced technology that can cure disease, end hunger, and reverse climate change. The world is waiting to see how this incredible offer will play out.',
             'A new study has found that eating pizza every day can lead to a longer life. The study, which was conducted by a team of Italian researchers, looked at the eating habits of over 10,000 people and found that those who ate pizza regularly lived on average two years longer than those who didn\'t. The study has been hailed as a breakthrough in the field of nutrition.',
             'In a surprising twist, a new study has found that smoking is actually good for you. The study, which was conducted by a team of British researchers, looked at the health outcomes of over 100,000 people and found that those who smoked regularly had lower rates of heart disease and cancer than those who didn\'t. The findings have sparked controversy among health experts.',
             'In a bizarre turn of events, the world\'s largest iceberg has been discovered in Florida. The iceberg, which is over 100 miles long and 50 miles wide, was found off the coast of Miami by a group of tourists on a whale-watching tour. Scientists are baffled by the discovery and are scrambling to figure out how an iceberg of this size could have']
})
fake_news_df

Begin by:
- converting the text to lowercase and splitting the text by whitespace
- adding a new column called `placeholder` with 1 as a placeholder value

In [ ]:
(
    fake_news_df
    .with_columns(
        pl.col("text").str.to_lowercase().str.split(" "),
        pl.lit(1).alias("placeholder")
    )
)

Explode the lists in the `text` column

In [ ]:
(
    fake_news_df
    .with_columns(
        pl.col("text").str.to_lowercase().str.split(" "),
        pl.lit(1).alias("placeholder")
    )
    .explode("text")
)

Pivot the output so that the article metadata is preserved on each row and the remainder of the columns indicate if the column name is present in the text of that article. Ensure the column names are sorted. 

In [ ]:
(
    fake_news_df
    .with_columns(
        pl.col("text").str.to_lowercase().str.split(" "),
        pl.lit(1).alias("placeholder")
    )
    .explode("text")
    .pivot(
        index=["publication","date","title"],
        on="text",
        values="placeholder",
        sort_columns=True,
        aggregate_function="max"
    )
)

Replace the `null` values with 0

In [ ]:
(
    fake_news_df
    .with_columns(
        pl.col("text").str.to_lowercase().str.split(" "),
        pl.lit(1).alias("placeholder")
    )
    .explode("text")
    .pivot(
        index=["publication","date","title"],
        on="text",
        values="placeholder",
        sort_columns=True,
        aggregate_function="max"

    )
    .fill_null(value=0)
)

If we wanted to split strings with a slightly more sophisticated pattern we could use the following regex (used by CountVectorizer in scikit-learn) and `str.extract_all`

In [ ]:
(
    fake_news_df
    .with_columns(
        pl.col("text").str.to_lowercase().str.extract_all('(?u)\\b\\w\\w+\\b'),
        pl.lit(1).alias("placeholder")
    )
    .explode("text")
    .pivot(
        index=["publication","date","title"],
        on="text",
        values="placeholder",
        sort_columns=True,
        aggregate_function="max"

    )
    .fill_null(value=0)
)

### Solution to exercise 3

We have a table showing electricity rates paid by a household. The rates can vary by:
- day of the week
- time of day with 00:00 overnight and 12:00 during the day

We identify each household with an `ID`. We use a single household in this exercise.

We store these rates in a table as follows

In [ ]:
pl.Config.set_tbl_rows(14)
df_rates = (
    pl.DataFrame(
        {
            "id":['A','A'],
            "Mon":[1,None],
            "Tue":[1,None],
            "Wed":[1,None],
            "Thu":[1,None],
            "Fri":[1,None],
            "Sat":[None,1],
            "Sun":[None,1],
            "00:00":[0.1,0.3],
            "12:00":[0.15,0.25]
            
        }
    )
)
df_rates

Use `unpivot` and `pivot` to transform `df_rates` to the following `DataFrame` where each row is a (day-of-week,time-of-day) pair and each column is the rate for each `id` in that period

In [ ]:
target_df = pl.DataFrame(
    [
        {"weekday": "Mon", "variable": "00:00", "A": 0.1},
        {"weekday": "Tue", "variable": "00:00", "A": 0.1},
        {"weekday": "Wed", "variable": "00:00", "A": 0.1},
        {"weekday": "Thu", "variable": "00:00", "A": 0.1},
        {"weekday": "Fri", "variable": "00:00", "A": 0.1},
        {"weekday": "Sat", "variable": "00:00", "A": 0.3},
        {"weekday": "Sun", "variable": "00:00", "A": 0.3},
        {"weekday": "Mon", "variable": "12:00", "A": 0.15},
        {"weekday": "Tue", "variable": "12:00", "A": 0.15},
        {"weekday": "Wed", "variable": "12:00", "A": 0.15},
        {"weekday": "Thu", "variable": "12:00", "A": 0.15},
        {"weekday": "Fri", "variable": "12:00", "A": 0.15},
        {"weekday": "Sat", "variable": "12:00", "A": 0.25},
        {"weekday": "Sun", "variable": "12:00", "A": 0.25},
    ]
)
target_df

First we `unpivot` the columns with the day of the week

In [ ]:
(
    df_rates
    .unpivot(
        index=["id","00:00","12:00"],variable_name="weekday"
    )
)

Then we remove the `null` values

In [ ]:
(
    df_rates
    .unpivot(
        index=["id","00:00","12:00"],variable_name="weekday"
    )
    .filter(
        pl.col("value").is_not_null()
    )
)

Now we can drop the `value` column (which we only needed for filtering) and `unpivot` the times of day

In [ ]:
(
    df_rates
    .unpivot(
        index=["id","00:00","12:00"],variable_name="weekday"
    )
    .filter(
        pl.col("value").is_not_null()
    )
    .drop("value")
    .unpivot(
        index=["id","weekday"]
    )
)

Finally we `pivot` the IDs to get them in a single column

In [ ]:
(
    df_rates
    .unpivot(
        index=["id","00:00","12:00"],variable_name="weekday"
    )
    .filter(
        pl.col("value").is_not_null()
    )
    .drop("value")
    .unpivot(
        index=["id","weekday"]
    )
    .pivot(
        index=["weekday","variable"],
        on="id",
        values="value"
    )
)